In [3]:
from app_settings import (
    DOCUMENT_DB_URI,
    AZURE_PRIMARY_KEY, AZURE_URL,
    GEMINI_API_KEY,
    connect_to_cosmosdb
)
from rag.indexing import insert_articles, EmbeddingService
from azure.cosmos import PartitionKey
import pandas as pd
import warnings 
warnings.filterwarnings("ignore")

ImportError: cannot import name 'DOCUMENT_DB_URI' from 'app_settings' (c:\Users\tiago\Documents\NOVA IMS\Data Science\3rd year\Fall\Capstone_Project\Project\deNoise\app_settings.py)

# Trying out RAG

## Chunking and Indexing

### Inserting the articles in the articles container

In [ ]:
news = pd.read_csv("news_database.csv")
news.head()

,id,title,text,date
0,19b1760e1f299ab5_1,"Stock markets December 13, 2025",Markets: Stocks slipped like your clumsiest fr...,2025-12-13 10:53:04
1,19b1760e1f299ab5_2,It’s not a stretch to say Lululemon needed a c...,During a mostly upbeat earnings call on Thursd...,2025-12-13 10:53:04
2,19b1760e1f299ab5_3,Pot stocks got high after reports Trump plans ...,Cannabis stocks rallied yesterday after news o...,2025-12-13 10:53:04
3,19b1760e1f299ab5_4,Executive order targets state AI laws,President Trump signed an executive order on T...,2025-12-13 10:53:04
4,19b1760e1f299ab5_5,Former Michigan football coach charged with cr...,Ex-University of Michigan football coach Sherr...,2025-12-13 10:53:04


In [ ]:
cosmosdb_client, deNoise_cosmosdb = connect_to_cosmosdb()

In [ ]:
for cont in deNoise_cosmosdb.list_containers():
    print(cont['id'])

newsArticles
userProfiles
newsEmbeddings


In [ ]:
newscontainer = deNoise_cosmosdb.get_container_client('newsArticles')

In [ ]:
len(list(newscontainer.read_all_items()))

669

### Embeddings

In [ ]:
text = list(newscontainer.read_all_items())[0]['title']
print(text)
type(text)

OpenAI, Amazon Sign $38 Billion Cloud Deal


str

In [ ]:
embeddingservice = EmbeddingService()
result = embeddingservice.generate_embedding(text=text)
result

[-0.0070019662,
 -0.0023241008,
 0.0044640778,
 -0.084099464,
 -0.011027054,
 0.004685113,
 0.039817106,
 0.019069528,
 -0.008995239,
 2.6935886e-05,
 -0.03661413,
 -0.0026772444,
 0.015470704,
 -0.007611877,
 0.109608755,
 0.011998819,
 0.014253483,
 -0.011594762,
 0.0155390175,
 0.0043275524,
 -0.010480972,
 -0.010693609,
 0.03183914,
 0.000721525,
 -0.0025996692,
 -0.008987109,
 0.026555033,
 -0.015239678,
 0.021596592,
 -0.008127287,
 -0.0048588375,
 -0.018267676,
 0.021259096,
 0.032728598,
 -0.010860972,
 0.028622821,
 0.02268366,
 -0.011431093,
 -0.0053429347,
 0.016993133,
 -0.0060531744,
 0.00951269,
 0.011424423,
 0.0051925275,
 -0.026201677,
 -0.022788472,
 0.0051345276,
 -0.019273147,
 -0.041176785,
 -0.0015840643,
 0.0032653955,
 0.001473156,
 -0.00054675917,
 -0.17107475,
 -0.005040998,
 0.012111169,
 0.0085012,
 0.0010916906,
 -0.008958267,
 0.0060346485,
 0.0052046534,
 0.020759093,
 -0.022948097,
 -0.0016393813,
 -0.026198698,
 -0.01891186,
 -0.0057785283,
 0.005322587

In [ ]:
results = embeddingservice.embed_documents(texts=[article['text'] for article in list(newscontainer.read_all_items())[:10]])
results

✅ Processed 10/10 embeddings
✅ Completed: 10/10 embeddings generated


[[0.0027072832,
  0.0011487498,
  0.0072176815,
  -0.06966001,
  -0.0055333897,
  0.0071876855,
  0.030943887,
  0.016082335,
  0.010407427,
  0.003083017,
  -0.017617293,
  0.020991115,
  0.025451707,
  -0.009460841,
  0.118922286,
  0.022543425,
  0.025001694,
  0.006421748,
  0.03213093,
  0.015801037,
  -0.0054936553,
  -0.00995614,
  0.022128595,
  0.0049265954,
  -0.0091924155,
  -0.0072244145,
  0.022609562,
  -0.005887547,
  0.020862626,
  -0.020182047,
  -0.014727929,
  -0.01626627,
  0.019369919,
  0.044537205,
  0.013661189,
  0.02308077,
  0.022346001,
  -0.012267383,
  0.01003225,
  0.016817223,
  0.001097821,
  -0.0024539891,
  0.009035611,
  -0.016897615,
  -0.021792023,
  -0.02659285,
  0.011525224,
  -0.024910372,
  -0.0196952,
  0.0032165472,
  -0.013740063,
  0.00075184146,
  0.0106532965,
  -0.16678189,
  -0.0008317927,
  0.004823209,
  0.011111902,
  -0.005270322,
  0.008293994,
  0.014416186,
  -0.006728737,
  0.03366766,
  -0.02095875,
  -0.02302042,
  -0.0210880

In [ ]:
embeddingscontainer = deNoise_cosmosdb.get_container_client('newsEmbeddings')

# Deleting the articles from the newsEmbeddings container

In [ ]:
def recreate_embeddings_container():
    """
    Delete and recreate the newsEmbeddings container.
    This is the fastest way to clear all data.
    """
    from app_settings import COSMOSDB_INDEXING_POLICY, COSMOSDB_VECTOR_EMBEDDING_POLICY
    
    cosmosdb_client, cosmos_db = connect_to_cosmosdb()
    
    # Delete the container
    try:
        cosmos_db.delete_container('newsEmbeddings')
        print("🗑️  Deleted newsEmbeddings container")
    except Exception as e:
        print(f"⚠️  Container might not exist: {e}")
    
    # Recreate the container
    partition_key = PartitionKey(path="/id")
    
    cosmos_db.create_container_if_not_exists(
        id='newsEmbeddings',
        partition_key=partition_key,
        indexing_policy=COSMOSDB_INDEXING_POLICY,
        vector_embedding_policy=COSMOSDB_VECTOR_EMBEDDING_POLICY
    )
    
    print("✅ Recreated newsEmbeddings container")


#recreate_embeddings_container()

🗑️  Deleted newsEmbeddings container
✅ Recreated newsEmbeddings container


# Index the articles

In [ ]:
cosmosdb_client, deNoise_cosmosdb = connect_to_cosmosdb()
newscontainer = deNoise_cosmosdb.get_container_client('newsArticles')

In [ ]:
len(list(deNoise_cosmosdb.get_container_client('newsEmbeddings').read_all_items()))

0

In [ ]:
len(list(newscontainer.read_all_items()))

669

In [ ]:
from rag.indexing import index_article
not_indexed_ids = []
all_articles = list(newscontainer.read_all_items())
current_indexed_count = 0

for article in all_articles:
    print(article['title'])
    try:
        index_article(
            contents=[article['title'], article['text']],
            article_title=article['title'],
            article_id=article['id'],
            article_date=article['date']
        )
        print("✅ Indexed successfully")
    
    except Exception as e:
        error_msg = str(e)
        print(f"❌ Error: {error_msg}")
        
        not_indexed_ids.append(article['id'])
        continue

# Save not indexed IDs to file
with open('not_indexed_articles.txt', 'w') as f:
    for article_id in not_indexed_ids:
        f.write(f"{article_id}\n")

print(f"\n📊 Summary:")
print(f"⏭️  Not indexed: {len(not_indexed_ids)} articles")
print(f"💾 IDs saved to 'not_indexed_articles.txt'")

OpenAI, Amazon Sign $38 Billion Cloud Deal
✅ Completed: 2/2 embeddings generated
✅ Indexed successfully
Facebook Dating Is a Surprise Hit for the Social Network
✅ Completed: 2/2 embeddings generated
✅ Indexed successfully
A commercial space station startup now has a foothold in space
✅ Completed: 2/2 embeddings generated
✅ Indexed successfully
Alphabet is increasingly launching ‘moonshot' projects as independent companies — here's why
✅ Completed: 2/2 embeddings generated
✅ Indexed successfully
Rmlx
✅ Completed: 2/2 embeddings generated
✅ Indexed successfully
The fetch()ening
✅ Completed: 2/2 embeddings generated
✅ Indexed successfully
The Great Decoupling of Labor and Capital
✅ Completed: 2/2 embeddings generated
✅ Indexed successfully
The Startup Winter Is Always Coming
✅ Completed: 2/2 embeddings generated
✅ Indexed successfully
An individual can change an organization
✅ Completed: 2/2 embeddings generated
✅ Indexed successfully
rm -rf remains
✅ Completed: 2/2 embeddings generated
✅

In [ ]:
from langchain_azure_ai.vectorstores import AzureCosmosDBNoSqlVectorSearch
from rag.indexing import EmbeddingService
from azure.cosmos import PartitionKey
from app_settings import (COSMOSDB_INDEXING_POLICY, COSMOSDB_VECTOR_SEARCH_FIELDS, COSMOSDB_VECTOR_EMBEDDING_POLICY)

title_to_find = "Dutch government takes control of Chinese-owned chipmaker Nexperia in 'highly exceptional' move"

embeddings = EmbeddingService()

def search_by_title_vector(article_title: str, query_text: str = None, k: int = 5) -> list:
    """
    Search vector database with title filter and optional semantic query.
    
    Args:
        article_title: Filter results to this article title
        query_text: Optional text to semantically search within the article
        k: Number of results to return
        
    Returns:
        List of matching documents with similarity scores
    """
    cosmosdb_client, cosmos_db = connect_to_cosmosdb()

    partition_key = PartitionKey(path="/id")
    cosmos_container_properties = {"partition_key": partition_key}

    # Initialize vector search
    vector_search = AzureCosmosDBNoSqlVectorSearch(
        cosmos_client=cosmosdb_client,
        embedding=embeddings,
        database_name='deNoise',
        container_name='vectorStore',
        vector_embedding_policy=COSMOSDB_VECTOR_EMBEDDING_POLICY,
        indexing_policy=COSMOSDB_INDEXING_POLICY,
        vector_search_fields=COSMOSDB_VECTOR_SEARCH_FIELDS,
        cosmos_database_properties={},
        cosmos_container_properties=cosmos_container_properties
    )
    
    # Search with metadata filter
    if query_text:
        # Semantic search within the article
        results = vector_search.similarity_search(
            query=query_text,
            k=k,
            filters={"title": article_title}
        )
    else:
        # Just retrieve chunks by title (using title as query)
        results = vector_search.similarity_search(
            query=article_title,
            k=k,
            filters={"title": article_title}
        )
    
    return results

search_by_title_vector('OpenAI and Amazon')

[Document(metadata={'title': 'OpenAI, Amazon Sign $38 Billion Cloud Deal', 'article_id': '19a4eb9435eaa367_2', 'date': '2025-11-04 11:55:54', 'is_title': True, 'id': '75724366-8ffd-4b09-b282-44d854d655c6'}, page_content='"OpenAI, Amazon Sign $38 Billion Cloud Deal"'),
 Document(metadata={'title': 'OpenAI, Amazon Sign $38 Billion Cloud Deal', 'article_id': '19a4eb9435eaa367_2', 'date': '2025-11-04 11:55:54', 'is_title': False, 'id': '875850ab-bd4b-4fc3-8b49-ffb1018386af'}, page_content='"OpenAI will pay Amazon $38 billion for computing power in a multiyear deal. Amazon expects all of the computing capacity negotiated as part of the agreement to be available by the end of the year. It wasn\'t able to sell its cloud services to OpenAI for years because OpenAI had an exclusive cloud-computing partnership with Microsoft. OpenAI is set to generate $13 billion in revenue this year. It will have to continue expanding sales at an exponential rate to pay for its computing needs. Sam Altman claim

### Testing similarity Search

In [ ]:
!pip install langchain-community

  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mypy_extensions-1.1.0-py3-none-any.whl.metadata (1.1 kB)
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 2.5/2.5 MB 24.2 MB/s  0:00:00
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 25.1 MB/s  0:00:00
Using cached marshmallow-3.26.1-py3-none-any.whl (50 kB)
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 2.1/2.1 MB 39.9 MB/s  0:00:00
Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
Using cached mypy_extensions-1.1.0-py3-none-any.whl (5.0 kB)

   --- ------------------------------------  1/

langchain-community==0.4.1

In [ ]:
import pandas as pd
from app_settings import connect_to_cosmosdb
from azure.cosmos import PartitionKey

def recreate_vectorstore_container():
    """
    Delete and recreate the newsEmbeddings container.
    This is the fastest way to clear all data.
    """
    from app_settings import COSMOSDB_INDEXING_POLICY, COSMOSDB_VECTOR_EMBEDDING_POLICY
    
    cosmosdb_client, cosmos_db = connect_to_cosmosdb()
    
    # Delete the container
    try:
        cosmos_db.delete_container('newsEmbeddings')
        print("🗑️  Deleted newsEmbeddings container")
    except Exception as e:
        print(f"⚠️  Container might not exist: {e}")
    
    # Recreate the container
    partition_key = PartitionKey(path="/id")
    
    cosmos_db.create_container_if_not_exists(
        id='newsEmbeddings',
        partition_key=partition_key,
        indexing_policy=COSMOSDB_INDEXING_POLICY,
        vector_embedding_policy=COSMOSDB_VECTOR_EMBEDDING_POLICY
    )
    
    print("✅ Recreated newsEmbeddings container")


#recreate_vectorstore_container()
#
#
#news = pd.read_csv("news_database.csv")[:100]
#display(news.head())
#from services.cosmos_db_service import CosmosDBService
#cosmos_service = CosmosDBService()
#
#for article in news.to_dict(orient='records'):
#    cosmos_service.index_article(
#        contents=[article['title'], article['text']],
#        article_title=article['title'],
#        article_id=article['id'],
#        article_date=article['date']
#    )
#    print(f"✅ Indexed successfully: {article['title']}")
#print("✅ All articles indexed.")

Project root path: C:\Users\tiago\Documents\NOVA IMS\Data Science\3rd year\Fall\Capstone_Project\Project\deNoise
🗑️  Deleted newsEmbeddings container
✅ Recreated newsEmbeddings container


,id,title,text,date
0,19b1760e1f299ab5_1,"Stock markets December 13, 2025",Markets: Stocks slipped like your clumsiest fr...,2025-12-13 10:53:04
1,19b1760e1f299ab5_2,It’s not a stretch to say Lululemon needed a c...,During a mostly upbeat earnings call on Thursd...,2025-12-13 10:53:04
2,19b1760e1f299ab5_3,Pot stocks got high after reports Trump plans ...,Cannabis stocks rallied yesterday after news o...,2025-12-13 10:53:04
3,19b1760e1f299ab5_4,Executive order targets state AI laws,President Trump signed an executive order on T...,2025-12-13 10:53:04
4,19b1760e1f299ab5_5,Former Michigan football coach charged with cr...,Ex-University of Michigan football coach Sherr...,2025-12-13 10:53:04


c:\Users\tiago\Documents\NOVA IMS\Data Science\3rd year\Fall\Capstone_Project\Project\deNoise\venv\Lib\site-packages\langchain_community\vectorstores\azure_cosmos_db_no_sql.py:266: UserWarning: Method 'from_texts' of AzureCosmosDBNoSql vector store invoked with unsupported arguments (vector_search_fields), which will be ignored.
  warnings.warn(


✅ Completed: 2/2 embeddings generated
✅ Indexed successfully: Stock markets December 13, 2025
✅ Completed: 2/2 embeddings generated
✅ Indexed successfully: It’s not a stretch to say Lululemon needed a change
✅ Completed: 2/2 embeddings generated
✅ Indexed successfully: Pot stocks got high after reports Trump plans to reclassify marijuana
✅ Completed: 2/2 embeddings generated
✅ Indexed successfully: Executive order targets state AI laws
✅ Completed: 2/2 embeddings generated
✅ Indexed successfully: Former Michigan football coach charged with crimes after firing
✅ Completed: 2/2 embeddings generated
✅ Indexed successfully: Appetizers boom in worrying sign for the economy
✅ Completed: 2/2 embeddings generated
✅ Indexed successfully: New photographs from the estate of the late disgraced financier Jeffrey Epstein were released by House Democrats yesterday, including some that feature President Trump, former President Bill Clinton, and Andrew Mountbatten-Windsor (formerly Prince Andrew).
✅ Co

In [2]:
from tools.choosing_rag import get_time_range_for_rag
from services.cosmos_db_service import CosmosDBService
start, end = get_time_range_for_rag('weekly')
print(start, end)
cosmos_service = CosmosDBService()
cosmos_service.get_time_range(start, end)

2025-12-14 2025-12-21


['2025-12-14',
 '2025-12-15',
 '2025-12-16',
 '2025-12-17',
 '2025-12-18',
 '2025-12-19',
 '2025-12-20',
 '2025-12-21']

In [4]:
start = '2025-12-11'
end = '2025-12-13'

In [2]:
from services.cosmos_db_service import CosmosDBService
cosmos_service = CosmosDBService()


results = cosmos_service.rag_retrieval(
    query="Zootopia 2 has made more than $1 billion, becoming the fastest PG movie to hit the box-office"
    #start_date=start,
    #end_date=end
)
print(results)

[]
No filters applied.
ARTICLE 1:
Zootopia 2 has made more than $1 billion, becoming the fastest PG movie to hit the box-office milestone (easy to do when you’re powered by a rabbit).


ARTICLE 2:
Disney to Invest $1 Billion in OpenAI and License Characters for Use in ChatGPT, Sora
Disney is investing $1 billion into OpenAI. The three-year licensing deal will allow OpenAI's users to generate videos with Disney's characters and properties in them. The agreement includes provisions that limit what Disney characters can do in generated videos. The partnership is the result of nearly two years of talks between the two companies. Disney is investing $1 billion into OpenAI. The three-year licensing deal will allow OpenAI's users to generate videos with Disney's characters and properties in them. The agreement includes provisions that limit what Disney characters can do in generated videos. The partnership is the result of nearly two years of talks between the two companies.

ARTICLE 3:
OpenA

In [5]:
query = 'Stock markets'
results = cosmos_service.rag_retrieval(
    query=query,
    start_date=start,
    end_date=end,
    k=20
)
print(results)

[Condition(property='metadata.date', operator='$eq', value='2025-12-11'), Condition(property='metadata.date', operator='$eq', value='2025-12-12'), Condition(property='metadata.date', operator='$eq', value='2025-12-13')]
Applying filters.
conditions=[Condition(property='metadata.date', operator='$eq', value='2025-12-11'), Condition(property='metadata.date', operator='$eq', value='2025-12-12'), Condition(property='metadata.date', operator='$eq', value='2025-12-13')] logical_operator='$or'
ARTICLE 1:
Stock markets December 13, 2025
Markets: Stocks slipped like your clumsiest friend wearing heels on an icy day yesterday, pulled down by investors’ fears of an AI bubble. Broadcom in particular plunged , losing $200 billion in market cap, after its earnings report sparked concerns about the profitability of its AI business.

ARTICLE 2:
It’s not a stretch to say Lululemon needed a change
During a mostly upbeat earnings call on Thursday, Lululemon announced that CEO Calvin McDonald would be ste

In [17]:
import json
def build_full_context(vector_results: list, container_client) -> str:
    """
    1. Extracts unique article IDs.
    2. Queries Cosmos DB for all parts (Title + Body).
    3. Robustly finds the text field (checking 'content', 'text', etc.).
    """
    
    # 1. Extract Unique Article IDs
    unique_ids = {doc.metadata['article_id'] for doc in vector_results}
    
    if not unique_ids:
        return "No articles found."

    # 2. Query DB
    ids_formatted = ", ".join([f"'{aid}'" for aid in unique_ids])
    query = f"SELECT * FROM c WHERE c.metadata.article_id IN ({ids_formatted})"
    
    full_docs = list(container_client.query_items(
        query=query,
        enable_cross_partition_query=True
    ))


    # 3. Organize Data
    articles_map = {}

    for item in full_docs:
        # Access nested metadata safely
        meta = item.get('metadata', {})
        art_id = meta.get('article_id')
        is_title = meta.get('is_title')
        
        if not art_id: 
            continue

        # ROBUST CONTENT FETCHING: Try 'content', then 'text', then 'pageContent'
        raw_content = item.get('text') 
        
        # Clean the content if it's a JSON string (from your indexing logic)
        try:
            if raw_content:
                clean_content = json.loads(raw_content)
            else:
                clean_content = "[Content Empty in DB]"
        except (TypeError, json.JSONDecodeError):
            clean_content = str(raw_content)

        # Initialize ID in map
        if art_id not in articles_map:
            articles_map[art_id] = {"title": None, "body": None}

        # Store in correct slot
        if is_title:
            articles_map[art_id]["title"] = clean_content
        else:
            articles_map[art_id]["body"] = clean_content

    # 4. Build Final String
    formatted_context = []
    
    for i, (art_id, parts) in enumerate(articles_map.items(), 1):
        title = parts["title"] if parts["title"] else "Title not found"
        body = parts["body"] if parts["body"] else ""
        
        entry = f"ARTICLE {i}:\n{title}\n{body}\n"
        formatted_context.append(entry)

    return "\n".join(formatted_context)

In [18]:
print(build_full_context(results, cosmos_service.vector_db))

ARTICLE 1:
Stock markets December 13, 2025
Markets: Stocks slipped like your clumsiest friend wearing heels on an icy day yesterday, pulled down by investors’ fears of an AI bubble. Broadcom in particular plunged , losing $200 billion in market cap, after its earnings report sparked concerns about the profitability of its AI business.

ARTICLE 2:
It’s not a stretch to say Lululemon needed a change
During a mostly upbeat earnings call on Thursday, Lululemon announced that CEO Calvin McDonald would be stepping down in January, leaving a position he’s held since 2018. Yesterday, investors cheered the news , sending the $100 yoga-pants company’s stock soaring by more than 9%. McDonald’s decision to move on came after more than a year of struggles and pressure from Chip Wilson, Lululemon’s founder and the company’s largest shareholder. Wilson took out a full-page ad in the Wall Street Journal in October expressing disapproval of the athleisure maker’s “loss of cool.” It’s a rare instance wh

In [1]:
from services.agents_service import AgentsService
agents_service = AgentsService()

Project root path: C:\Users\tiago\Documents\NOVA IMS\Data Science\3rd year\Fall\Capstone_Project\Project\deNoise


In [ ]:
response = agents_service.generate_chat_answer(
    prompt="Give me a summary of the news about trump in this news page: https://tldr.tech/",
    user_id='a2xmZXJyZWlyYTI4QGdt'
)

Project root path: C:\Users\tiago\Documents\NOVA IMS\Data Science\3rd year\Fall\Capstone_Project\Project\deNoise


Invalid as_type 'retrieval'. Valid types are: agent, chain, embedding, evaluator, generation, guardrail, retriever, span, tool. Defaulting to 'span'.


In [3]:
print(response)

In September 2025, American Bitcoin, co-founded by Eric Trump and backed by Donald Trump Jr., saw a nearly 17% jump to $8.04 per share on its Nasdaq debut. Earlier that year, President Trump announced a $500 billion AI joint venture named Stargate, involving OpenAI, Oracle, SoftBank, and other partners. His administration also removed the Biden-era AI chip export controls. Additionally, he mentioned a new US sovereign-wealth fund potentially acquiring TikTok and granted a second 75-day extension to the app's forced-sale deadline, which TikTok attributed to its continued operation.


In [4]:
report = agents_service.generate_report(
    topics='AI, Climate Change and Stock Markets',
    time_range='monthly',
    structure='very concise, only key points, no elaboration, no explanations',
    user_id='a2xmZXJyZWlyYTI4QGdt'
)

c:\Users\tiago\Documents\NOVA IMS\Data Science\3rd year\Fall\Capstone_Project\Project\deNoise\services\cosmos_db_service.py:206: LangChainDeprecationWarning: The class `AzureCosmosDBNoSqlVectorSearch` was deprecated in LangChain 0.3.22 and will be removed in 1.0. An updated version of the class exists in the `langchain-azure-ai package and should be used instead. To use it run `pip install -U `langchain-azure-ai` and import as `from `langchain_azure_ai.vectorstores import AzureCosmosDBNoSqlVectorSearch``.
  vector_search = AzureCosmosDBNoSqlVectorSearch(


[Condition(property='metadata.date', operator='$eq', value='2025-11-21'), Condition(property='metadata.date', operator='$eq', value='2025-11-22'), Condition(property='metadata.date', operator='$eq', value='2025-11-23'), Condition(property='metadata.date', operator='$eq', value='2025-11-24'), Condition(property='metadata.date', operator='$eq', value='2025-11-25'), Condition(property='metadata.date', operator='$eq', value='2025-11-26'), Condition(property='metadata.date', operator='$eq', value='2025-11-27'), Condition(property='metadata.date', operator='$eq', value='2025-11-28'), Condition(property='metadata.date', operator='$eq', value='2025-11-29'), Condition(property='metadata.date', operator='$eq', value='2025-11-30'), Condition(property='metadata.date', operator='$eq', value='2025-12-01'), Condition(property='metadata.date', operator='$eq', value='2025-12-02'), Condition(property='metadata.date', operator='$eq', value='2025-12-03'), Condition(property='metadata.date', operator='$eq'

In [5]:
print(report)

**Market Intelligence Report: AI and Stock Markets**

**Date:** December 13, 2025

**Key Takeaways:**
*   **AI Concerns:** Investor fears regarding an "AI bubble" and the profitability of AI businesses significantly impacted stock markets, particularly on December 13, 2025, with Broadcom experiencing a substantial market cap decline.
*   **Interest Rate Influence:** The Federal Reserve's decision to lower interest rates on December 11, 2025, provided a temporary boost to markets, aligning with investor expectations.
*   **Market Volatility:** The week saw considerable market fluctuations, driven by a mix of monetary policy decisions and evolving sentiment around AI sector performance.

---

## Market Overview (December 10-13, 2025)

The stock markets experienced a volatile period between December 10 and December 13, 2025, primarily influenced by monetary policy decisions and growing investor apprehension concerning the artificial intelligence (AI) sector.

On **December 10, 2025**, mar

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
podcast = agents_service.generate_podcast(
    topics='AI, Climate Change and Stock Markets',
    time_range = 'monthly',
    structure = 'concise, engaging, suitable for audio format',
    user_id='a2xmZXJyZWlyYTI4QGdt'
)

c:\Users\tiago\Documents\NOVA IMS\Data Science\3rd year\Fall\Capstone_Project\Project\deNoise\services\cosmos_db_service.py:205: LangChainDeprecationWarning: The class `AzureCosmosDBNoSqlVectorSearch` was deprecated in LangChain 0.3.22 and will be removed in 1.0. An updated version of the class exists in the `langchain-azure-ai package and should be used instead. To use it run `pip install -U `langchain-azure-ai` and import as `from `langchain_azure_ai.vectorstores import AzureCosmosDBNoSqlVectorSearch``.
  vector_search = AzureCosmosDBNoSqlVectorSearch(


[Condition(property='metadata.date', operator='$eq', value='2025-11-21'), Condition(property='metadata.date', operator='$eq', value='2025-11-22'), Condition(property='metadata.date', operator='$eq', value='2025-11-23'), Condition(property='metadata.date', operator='$eq', value='2025-11-24'), Condition(property='metadata.date', operator='$eq', value='2025-11-25'), Condition(property='metadata.date', operator='$eq', value='2025-11-26'), Condition(property='metadata.date', operator='$eq', value='2025-11-27'), Condition(property='metadata.date', operator='$eq', value='2025-11-28'), Condition(property='metadata.date', operator='$eq', value='2025-11-29'), Condition(property='metadata.date', operator='$eq', value='2025-11-30'), Condition(property='metadata.date', operator='$eq', value='2025-12-01'), Condition(property='metadata.date', operator='$eq', value='2025-12-02'), Condition(property='metadata.date', operator='$eq', value='2025-12-03'), Condition(property='metadata.date', operator='$eq'

In [3]:
print(podcast)

Hey everyone, welcome to The deNoise Signal! I'm your host, and we're cutting through the noise on market volatility.

Our first segment, Market Movers. AI bubble fears are back, big time. Broadcom plunged yesterday, losing a staggering $200 billion in market cap. Its earnings report raised serious questions about AI business profitability. This follows earlier AI overspending concerns, though investors did buy that dip. On a brighter note, the Fed lowered interest rates, sending stocks surging. But Rivian also made waves, declining after announcing it's ditching Nvidia chips for its own. That's a big shift!

That's all for today's deNoise Signal. Stay sharp, and we'll catch you next time!
